In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData, Object,storing
import random

In [2]:
def classification(objectclass, Trainingclass, prediction):
    ## Star, Quasar, Galaxy, BAL
    classi = []
    ##Location of object that is predicted to be a given classification
    loc = [0,0,0,0]
    star=0;
    #quasar w/ redshift <2.1
    qso=0;
    gal=0;
    #quasar w/ redshift >2.1
    bal=0;
    starloc=[];
    qsoloc=[];
    galloc=[];
    balloc=[];
    i=0
    while i< len(Trainingclass):
        currentobject = Trainingclass[i]
        currentpred = prediction[i]
        if objectclass==currentobject:
            if currentpred==1: 
                star=star+1
                starloc.append(i)
            elif currentpred==3: 
                qso=qso+1
                qsoloc.append(i)
            elif currentpred==4: 
                gal=gal+1
                galloc.append(i)
            elif currentpred==30: 
                bal=bal+1
                balloc.append(i)
        
        i=i+1
    
    classi.append(star/(star+qso+gal+bal))
    classi.append(qso/(star+qso+gal+bal))
    classi.append(gal/(star+qso+gal+bal))
    classi.append(bal/(star+qso+gal+bal))
    return classi,starloc,qsoloc,galloc,balloc

In [3]:
bin_size = np.str(10)
TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")
PlateDir ="D:"+slash+"Data"+slash+"FullRebin10"+".txt"
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()

In [4]:
wav_log=[]
Full_table = []
PLATEIDs = []
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir+slash+'rebintrain'+slash+spectrum ,memmap=True)
    primhdu_ = plate_[0]
    hdu =  plate_[1].data
    Full_table.append(hdu)
    PLATEIDs.append(primhdu_.header['Plate'])
    wav_log.append(primhdu_.header['LogWav'])

In [5]:
plate_r  = []
plate_n  = []
plate_c  = []
plate_x  = []
plate_y  = []
i = 0
while i < len(PLATEIDs):
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_y = currentobj[2]
        Currentplate_x = currentobj[1]
        Currentplate_n = currentobj[0]
        plate_r.append(Currentplate_z)
        plate_n.append(Currentplate_n)
        plate_x.append(Currentplate_x[:800])
        plate_y.append(Currentplate_y)
        obj = obj +1
    i = i+1



In [6]:
hiddenlayer_format = (13)
backprop_method = 'lbfgs'
lr=0.00001
act =  'tanh'#'tanh'
mlp = MLPClassifier(hidden_layer_sizes=hiddenlayer_format,max_iter=500, solver = backprop_method,learning_rate_init=lr,activation=act) ##Think About

In [7]:
len(plate_x[3])

800

In [68]:
scaler = StandardScaler()  
scaler.fit(np.array(plate_x))  
plate_x = scaler.transform(plate_x)    
depth=80
estimators =20
rf = RandomForestClassifier(n_estimators =estimators ,max_depth=depth, random_state=0)
rf.fit(plate_x,plate_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [69]:
TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")
PlateDir ="D:"+slash+"Data"+slash+"FullTest10.txt"
with open(PlateDir) as f:
    Spectra_TestFiles = f.read().splitlines()

In [70]:
wav_log_Test=[]
Full_table_Test = []
PLATEID_Test = []
for spectrum in Spectra_TestFiles:
    plate_test = fits.open( TrainingDir+slash+'rebin'+slash+spectrum ,memmap=True)
    primhdu_test = plate_[0]
    hdu =  plate_test[1].data
    Full_table_Test.append(hdu)
    PLATEID_Test.append(primhdu_.header['Plate'])
    wav_log_Test.append(primhdu_.header['LogWav'])

In [71]:
plate_rt = []
plate_nt = []
plate_ct = []
plate_xt = []
plate_yt = []
i = 0
while i < len(PLATEID_Test):
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_y = currentobj[2]
        Currentplate_x = currentobj[1]
        Currentplate_n = currentobj[0]
        plate_rt.append(Currentplate_z)
        plate_nt.append(Currentplate_n)
        plate_xt.append(Currentplate_x[:800])
        plate_yt.append(Currentplate_y)
        obj = obj +1
    i = i+1



In [72]:
predictions = rf.predict(np.array(plate_xt))

In [73]:
Y_test=plate_yt
print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

          1       0.82      0.56      0.66       745
          3       0.33      0.62      0.43       291
          4       0.50      0.08      0.14        51
         30       0.68      0.72      0.70       658

avg / total       0.68      0.61      0.62      1745



In [74]:
star,star_starloc,star_lowzloc,star_galloc,star_highzloc = classification(1,Y_test,predictions) 
lowz,lowz_starloc,lowz_loc,lowz_galloc,lowz_highzloc = classification(3,Y_test,predictions)
gal,gal_starloc,gal_lowzloc,gal_galloc,gal_highzloc = classification(4,Y_test,predictions)
highz,highz_starloc,highz_lowzloc,highz_galloc,highz_highzloc = classification(30,Y_test,predictions)

In [75]:
File_Name = input("Please Enter File name: ")

Please Enter File name: 8020


In [76]:
d = open(TrainingDir+slash+File_Name+"rebin"+bin_size+"RF.txt", 'w')

In [77]:
t1=["Number of training objects: ",np.str(len(plate_y)), "\n"]
sp= "\n"
t2 = ["Numbr of testing: ",np.str(len(Y_test)), "\n"]
t3 = ["Number of estimators = ",np.str(len(Spectra_Files)), "\n"]
t5 = ["Depth = ",np.str(depth), "\n","\n"]
t4 = ["Bin Size = ",bin_size, "\n"]
n1 = ["Structure of neural network: ", np.str(hiddenlayer_format),"\n"]
n2  = ["Backpropagation method used: ",np.str(backprop_method), "\n"]
n3  = ["Learning rate: ",np.str(lr), "\n"]
n4  = ["Activation Function: ",np.str(act), "\n"]
r1 = ["Results of Neural Network: ", "\n","\n"]
r2=["        ","       Star    z<2.1   Galaxy   z>2.1 ","\n",]
r3="Star           ",np.str(np.round(star[0]*100,2)),"%  ", np.str(np.round(star[1]*100,2)),"%  ",np.str(np.round(star[2]*100,2)),"%  ",np.str(np.round(star[3]*100,2)),"%","\n"
r4="Quasar z<2.1   ",np.str(np.round( lowz[0]*100,2)),"%  ", np.str(np.round( lowz[1]*100,2)),"%  ",np.str(np.round( lowz[2]*100,2)),"%  ",np.str(np.round( lowz[3]*100,2)),"%","\n"
r5="Galaxy         ",np.str(np.round( gal[0]*100,2)),"%  ", np.str(np.round( gal[1]*100,2)),"%  ",np.str(np.round( gal[2]*100,2)),"%  ",np.str(np.round( gal[3]*100,2)),"%","\n"
r6="Quasar z>2.1   ",np.str(np.round( highz[0]*100,2)),"%  ", np.str(np.round( highz[1]*100,2)),"%  ",np.str(np.round( highz[2]*100,2)),"%  ",np.str(np.round (highz[3]*100,2)),"%","\n"

In [78]:
d.writelines(t1)
d.writelines(sp)
d.writelines(t2)
d.writelines(sp)
d.writelines(t3)
d.writelines(t4)
d.writelines(t5)
d.writelines(sp)
d.writelines(sp)
d.writelines(n1)
d.writelines(n2)
d.writelines(n3)
d.writelines(n4)
d.writelines(sp)
d.writelines(sp)
d.writelines(r1)
d.writelines(r2)
d.writelines(r3)
d.writelines(r4)
d.writelines(r5)
d.writelines(r6)

In [79]:
d.close()